In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import pairwise_distances
import pandas as pd

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

movies = pd.read_csv("dataset/movies.csv",encoding="Latin1")
Ratings = pd.read_csv("dataset/ratings.csv")
Tags = pd.read_csv("dataset/tags.csv",encoding="Latin1")


display(movies.head())
display(Ratings.head())
display(Tags.head())

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


,userId,movieId,rating,timestamp
0,12882,1,4.0,1147195252
1,12882,32,3.5,1147195307
2,12882,47,5.0,1147195343
3,12882,50,5.0,1147185499
4,12882,110,4.5,1147195239


,movieId,userId,tag,timestamp
0,3916,12882,sports,1147195545
1,4085,12882,Eddie Murphy,1147195966
2,33660,12882,boxing,1147195514
3,1197,320,must show,1145964801
4,1396,320,must show,1145964810


In [15]:
#Mean of all user ratings
Mean = Ratings.groupby(by="userId",as_index=False)['rating'].mean()
#Merging of Rating and Mean table, that will store mean of all user rating of particular user
Rating_avg = pd.merge(Ratings,Mean,on='userId')
#Difference of user mean rating and user rating for particular movie
Rating_avg['adg_rating']=Rating_avg['rating_x']-Rating_avg['rating_y']

display(Mean.head())
display(Rating_avg.head())


,userId,rating
0,316,3.329457
1,320,3.701613
2,359,3.685474
3,370,3.794404
4,910,3.898080


,userId,movieId,rating_x,timestamp,rating_y,adg_rating
0,12882,1,4.0,1147195252,4.061321,-0.061321
1,12882,32,3.5,1147195307,4.061321,-0.561321
2,12882,47,5.0,1147195343,4.061321,0.938679
3,12882,50,5.0,1147185499,4.061321,0.938679
4,12882,110,4.5,1147195239,4.061321,0.438679


In [17]:
#Make user movie table with actual rating of each user as value
check = pd.pivot_table(Rating_avg,values='rating_x',index='userId',columns='movieId')
display(check.head())
#make user movie  with mean_rating-user_rating of each user
final = pd.pivot_table(Rating_avg,values='adg_rating',index='userId',columns='movieId')
display(final.head())

movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,2.5,NaN,NaN,NaN,NaN,NaN,2.0,NaN,2.5,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,5.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,4.0,4.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,4.5,4.0,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,2.5,3.0,4.5,4.0,NaN,NaN,3.0,4.5,3.5,3.0
910,5.0,4.0,3.5,NaN,3.5,3.5,NaN,NaN,NaN,4.0,...,NaN,NaN,3.5,NaN,NaN,NaN,NaN,4.5,NaN,NaN


movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,NaN,NaN,NaN,NaN,NaN,-1.329457,NaN,-0.829457,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
320,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
359,1.314526,NaN,NaN,NaN,NaN,1.314526,NaN,NaN,0.314526,0.314526,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
370,0.705596,0.205596,NaN,NaN,NaN,1.205596,NaN,NaN,NaN,NaN,...,-1.294404,-0.794404,0.705596,0.205596,NaN,NaN,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.39808,NaN,-0.39808,-0.398080,NaN,NaN,NaN,0.101920,...,NaN,NaN,-0.398080,NaN,NaN,NaN,NaN,0.601920,NaN,NaN


In [22]:
# Replacing NaN by Movie Average
final_movie = final.fillna(final.mean(axis=0))

# Replacing NaN by user Average
final_user = final.apply(lambda row: row.fillna(row.mean()), axis=1)

display(final_movie.head())
display(final_user.head())


movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,-0.829457,-0.436518,-0.468109,-0.770223,-0.615331,0.320415,-1.329457,-0.690175,-0.829457,-0.094277,...,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
320,0.200220,-0.436518,-0.468109,-0.770223,-0.615331,0.320415,-0.203889,-0.690175,-0.150642,-0.094277,...,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
359,1.314526,-0.436518,-0.468109,-0.770223,-0.615331,1.314526,-0.203889,-0.690175,0.314526,0.314526,...,0.105075,0.006629,0.262314,0.237350,0.429868,0.306567,0.225110,0.234458,0.362468,0.349157
370,0.705596,0.205596,-0.468109,-0.770223,-0.615331,1.205596,-0.203889,-0.690175,-0.150642,-0.094277,...,-1.294404,-0.794404,0.705596,0.205596,0.429868,0.306567,-0.794404,0.705596,-0.294404,-0.794404
910,1.101920,0.101920,-0.398080,-0.770223,-0.398080,-0.398080,-0.203889,-0.690175,-0.150642,0.101920,...,0.105075,0.006629,-0.398080,0.237350,0.429868,0.306567,0.225110,0.601920,0.362468,0.349157


movieId,1,2,3,4,5,6,7,9,10,11,...,106487,106489,106782,106920,109374,109487,111362,111759,112556,112852
userId,,,,,,,,,,,,,,,,,,,,,
316,-8.294574e-01,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.329457e+00,-1.445872e-16,-8.294574e-01,-1.445872e-16,...,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16,-1.445872e-16
320,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,...,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17,-5.013910e-17
359,1.314526e+00,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,1.314526e+00,9.809414e-17,9.809414e-17,3.145258e-01,3.145258e-01,...,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17,9.809414e-17
370,7.055961e-01,2.055961e-01,-1.728814e-16,-1.728814e-16,-1.728814e-16,1.205596e+00,-1.728814e-16,-1.728814e-16,-1.728814e-16,-1.728814e-16,...,-1.294404e+00,-7.944039e-01,7.055961e-01,2.055961e-01,-1.728814e-16,-1.728814e-16,-7.944039e-01,7.055961e-01,-2.944039e-01,-7.944039e-01
910,1.101920e+00,1.019202e-01,-3.980798e-01,-1.994138e-16,-3.980798e-01,-3.980798e-01,-1.994138e-16,-1.994138e-16,-1.994138e-16,1.019202e-01,...,-1.994138e-16,-1.994138e-16,-3.980798e-01,-1.994138e-16,-1.994138e-16,-1.994138e-16,-1.994138e-16,6.019202e-01,-1.994138e-16,-1.994138e-16


In [25]:

# user similarity on replacing NAN by user avg
b = cosine_similarity(final_user)
display(b)
np.fill_diagonal(b, 0 )
similarity_with_user = pd.DataFrame(b,index=final_user.index)
similarity_with_user.columns=final_user.index
display(similarity_with_user.head())

array([[ 1.00000000e+00,  6.00630917e-02,  7.20754753e-02, ...,
         7.73765811e-02,  2.69240918e-02, -2.27270553e-02],
       [ 6.00630917e-02,  1.00000000e+00,  6.30541209e-02, ...,
         9.88451426e-02,  3.87524588e-02,  5.66393007e-02],
       [ 7.20754753e-02,  6.30541209e-02,  1.00000000e+00, ...,
         1.02698453e-01,  9.93065563e-02,  3.14714649e-03],
       ...,
       [ 7.73765811e-02,  9.88451426e-02,  1.02698453e-01, ...,
         1.00000000e+00,  9.49195502e-02,  6.47542404e-02],
       [ 2.69240918e-02,  3.87524588e-02,  9.93065563e-02, ...,
         9.49195502e-02,  1.00000000e+00,  8.99631810e-04],
       [-2.27270553e-02,  5.66393007e-02,  3.14714649e-03, ...,
         6.47542404e-02,  8.99631810e-04,  1.00000000e+00]])

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.060063,0.072075,0.043266,0.039305,0.045616,0.035341,0.038068,-1.248514e-02,0.050183,...,0.052632,0.104864,0.011358,0.029674,0.092552,0.017876,0.051371,0.077377,0.026924,-0.022727
320,0.060063,0.000000,0.063054,0.027315,0.006811,0.075620,0.011910,0.042509,-1.810374e-31,0.067389,...,0.115325,0.065130,0.071996,0.097554,0.064769,-0.006251,0.077256,0.098845,0.038752,0.056639
359,0.072075,0.063054,0.000000,0.135836,0.076131,0.036757,0.046418,0.066544,4.287659e-02,0.109726,...,0.120191,0.020672,0.032166,0.039599,0.108502,0.026371,0.075492,0.102698,0.099307,0.003147
370,0.043266,0.027315,0.135836,0.000000,0.108404,0.071655,0.070893,-0.003139,5.223516e-02,0.090241,...,0.091218,0.049594,0.004344,0.040692,0.110434,0.019767,-0.001364,0.052187,0.050997,0.009950
910,0.039305,0.006811,0.076131,0.108404,0.000000,0.021814,0.027339,-0.032211,-6.301121e-03,-0.007491,...,0.039464,-0.017620,0.020058,-0.004581,0.040866,-0.001438,-0.026082,0.073272,-0.012058,0.007610


In [27]:
# user similarity on replacing NAN by item(movie) avg
cosine = cosine_similarity(final_movie)
np.fill_diagonal(cosine, 0 )
similarity_with_movie = pd.DataFrame(cosine,index=final_movie.index)
similarity_with_movie.columns=final_user.index
display(similarity_with_movie)

userId,316,320,359,370,910,975,1015,1387,1447,1588,...,137118,137209,137227,137446,137559,137609,137805,138072,138176,138200
userId,,,,,,,,,,,,,,,,,,,,,
316,0.000000,0.921169,0.665659,0.673486,0.694247,0.894969,0.805780,0.851492,0.945224,0.705491,...,0.827564,0.895641,0.879290,0.916856,0.912146,0.922262,0.587738,0.671783,0.949138,0.740220
320,0.921169,0.000000,0.687225,0.691158,0.699527,0.916020,0.816931,0.874283,0.970234,0.724147,...,0.861798,0.909376,0.907009,0.938964,0.929049,0.943265,0.612746,0.695382,0.973853,0.768459
359,0.665659,0.687225,0.000000,0.534369,0.523475,0.655225,0.602806,0.629143,0.705042,0.542504,...,0.621820,0.654320,0.655839,0.679696,0.683900,0.686193,0.418283,0.489595,0.707370,0.534065
370,0.673486,0.691158,0.534369,0.000000,0.547560,0.671810,0.618456,0.628825,0.712683,0.548592,...,0.636688,0.673489,0.651209,0.688647,0.689265,0.692595,0.405881,0.497332,0.714011,0.546637
910,0.694247,0.699527,0.523475,0.547560,0.000000,0.680701,0.621463,0.634921,0.723574,0.528281,...,0.638257,0.668887,0.677377,0.701964,0.701245,0.705041,0.408456,0.509008,0.725896,0.554105
975,0.894969,0.916020,0.655225,0.671810,0.680701,0.000000,0.787005,0.841706,0.934909,0.684281,...,0.836673,0.878770,0.865374,0.908539,0.893550,0.910820,0.578943,0.672592,0.938530,0.737176
1015,0.805780,0.816931,0.602806,0.618456,0.621463,0.787005,0.000000,0.754730,0.840091,0.659660,...,0.731160,0.786704,0.781250,0.815294,0.809223,0.814019,0.520639,0.583422,0.842743,0.680044
1387,0.851492,0.874283,0.629143,0.628825,0.634921,0.841706,0.754730,0.000000,0.888543,0.654423,...,0.799955,0.841136,0.830029,0.860272,0.848434,0.867857,0.557445,0.633681,0.893887,0.718463
1447,0.945224,0.970234,0.705042,0.712683,0.723574,0.934909,0.840091,0.888543,0.000000,0.742589,...,0.875640,0.928122,0.921501,0.957073,0.950796,0.964507,0.619789,0.705041,0.993466,0.780233


In [28]:
def find_n_neighbours(df,n):
    order = np.argsort(df.values, axis=1)[:, :n]
    df = df.apply(lambda x: pd.Series(x.sort_values(ascending=False)
           .iloc[:n].index, 
          index=['top{}'.format(i) for i in range(1, n+1)]), axis=1)
    return df
# top 30 neighbours for each user
sim_user_30_u = find_n_neighbours(similarity_with_user,30)
display(sim_user_30_u.head())

,top1,top2,top3,top4,top5,top6,top7,top8,top9,top10,...,top21,top22,top23,top24,top25,top26,top27,top28,top29,top30
userId,,,,,,,,,,,,,,,,,,,,,
316,113673,117918,9050,12882,38187,102668,98880,43829,13215,78501,...,88608,120782,74472,53834,88928,42245,58265,89527,49830,63902
320,12288,113673,28159,79846,134627,112948,120729,97163,2945,4931,...,39271,94883,127683,101137,54989,134521,80946,10055,64365,106512
359,102118,96482,102532,50898,2702,60016,23428,120782,57937,42096,...,117258,7723,120729,61305,40768,117918,86768,129498,131620,58346
370,46645,42245,40768,23428,123707,60016,45120,113645,97195,102118,...,5611,20530,2702,38159,359,43354,117144,96482,2988,108195
910,87042,131620,67352,40768,31321,48821,26222,63295,5611,370,...,134521,88738,46645,108195,70201,58265,18115,114601,23428,17039


In [31]:
sim_user_30_m = find_n_neighbours(similarity_with_movie,30)
sim_user_30_m.head()

def get_user_similar_movies( user1, user2 ):
    common_movies = Rating_avg[Rating_avg.userId == user1].merge(
    Rating_avg[Rating_avg.userId == user2],
    on = "movieId",
    how = "inner" )
    return common_movies.merge( movies, on = 'movieId' )

a = get_user_similar_movies(370,86309)
a = a.loc[ : , ['rating_x_x','rating_x_y','title']]
display(a.head())

,rating_x_x,rating_x_y,title
0,5.0,5.0,"Matrix, The (1999)"
1,5.0,4.5,"Lord of the Rings: The Fellowship of the Ring,..."
2,5.0,4.0,"Lord of the Rings: The Two Towers, The (2002)"
3,4.5,4.0,"Lord of the Rings: The Return of the King, The..."
4,1.5,1.0,Serenity (2005)


In [33]:
def User_item_score(user,item):
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    c = final_movie.loc[:,item]
    d = c[c.index.isin(b)]
    f = d[d.notnull()]
    avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
    index = f.index.values.squeeze().tolist()
    corr = similarity_with_movie.loc[user,index]
    fin = pd.concat([f, corr], axis=1)
    fin.columns = ['adg_score','correlation']
    fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
    nume = fin['score'].sum()
    deno = fin['correlation'].sum()
    final_score = avg_user + (nume/deno)
    return final_score

score = User_item_score(320,7371)
print("score (u,i) is",score)


Rating_avg = Rating_avg.astype({"movieId": str})
Movie_user = Rating_avg.groupby(by = 'userId')['movieId'].apply(lambda x:','.join(x))


def User_item_score1(user):
    Movie_seen_by_user = check.columns[check[check.index==user].notna().any()].tolist()
    a = sim_user_30_m[sim_user_30_m.index==user].values
    b = a.squeeze().tolist()
    d = Movie_user[Movie_user.index.isin(b)]
    l = ','.join(d.values)
    Movie_seen_by_similar_users = l.split(',')
    Movies_under_consideration = list(set(Movie_seen_by_similar_users)-set(list(map(str, Movie_seen_by_user))))
    Movies_under_consideration = list(map(int, Movies_under_consideration))
    score = []
    for item in Movies_under_consideration:
        c = final_movie.loc[:,item]
        d = c[c.index.isin(b)]
        f = d[d.notnull()]
        avg_user = Mean.loc[Mean['userId'] == user,'rating'].values[0]
        index = f.index.values.squeeze().tolist()
        corr = similarity_with_movie.loc[user,index]
        fin = pd.concat([f, corr], axis=1)
        fin.columns = ['adg_score','correlation']
        fin['score']=fin.apply(lambda x:x['adg_score'] * x['correlation'],axis=1)
        nume = fin['score'].sum()
        deno = fin['correlation'].sum()
        final_score = avg_user + (nume/deno)
        score.append(final_score)
    data = pd.DataFrame({'movieId':Movies_under_consideration,'score':score})
    top_5_recommendation = data.sort_values(by='score',ascending=False).head(5)
    Movie_Name = top_5_recommendation.merge(movies, how='inner', on='movieId')
    Movie_Names = Movie_Name.title.values.tolist()
    return Movie_Names

user = int(input("Enter the user id to whom you want to recommend : "))
predicted_movies = User_item_score1(user)
print(" ")
print("The Recommendations for User Id : 370")
print("   ")
for i in predicted_movies:
    print(i)

score (u,i) is 4.255766437391595
Enter the user id to whom you want to recommend : 370
 
The Recommendations for User Id : 370
   
Band of Brothers (2001)
Godfather: Part II, The (1974)
Wallace & Gromit: The Wrong Trousers (1993)
Bicycle Thieves (a.k.a. The Bicycle Thief) (a.k.a. The Bicycle Thieves) (Ladri di biciclette) (1948)
Spirited Away (Sen to Chihiro no kamikakushi) (2001)
